# Multi threading

In [8]:
import threading
import time

## 1. add thread

In [7]:

def thread_job():
    print('this is an added thread, number is {}'.format(threading.current_thread()))


def main():
    # define thread
    added_thread = threading.Thread(target=thread_job)
    # run thread
    added_thread.start()
    
    # print num of thread
    print(threading.active_count())
    # list all thread
    print(threading.enumerate())
    
    print(threading.current_thread())
if __name__ == '__main__':
    main()

this is an added thread, number is <Thread(Thread-4, started 140170313983744)>
5
[<_MainThread(MainThread, started 140170808362752)>, <Thread(Thread-2, started daemon 140170730907392)>, <Heartbeat(Thread-3, started daemon 140170722514688)>, <HistorySavingThread(IPythonHistorySavingThread, started 140170697336576)>, <ParentPollerUnix(Thread-1, started daemon 140170351343360)>]
<_MainThread(MainThread, started 140170808362752)>


## 2. join

### 2.1 no join

In [9]:
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')

def main():
    # define thread
    added_thread = threading.Thread(target=thread_job, name='T1')
    # run thread
    added_thread.start()
    print('all done\n')

if __name__ == '__main__':
    main()

T1 start

all done

T1 finish



### 2.2 join

join 之后的命令都要等到join的线程运行完才会运行的运行完成才会运行

In [13]:
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')

def T2_job():
    print('T2 start\n')
    print('T2 finish\n')
def main():
    # define thread
    added_thread = threading.Thread(target=thread_job, name='T1')
    
    thread2 = threading.Thread(target=T2_job, name='T2')
    # run thread
    added_thread.start()
    thread2.start()
    added_thread.join()
    thread2.join()
    print('all done\n')

if __name__ == '__main__':
    main()

T1 start

T2 start

T2 finish

T1 finish

all done



## 3 Queue
threading的运行结果没有返回值，需要把结果放在队列中

In [14]:
from queue import Queue

In [20]:
def job(l,q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)

def multithreading(data=[[1,2,3],[3,4,5],[4,4,4],[5,5,5]]):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(data[i],q))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)
        
                            
if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


## 4. GIL
多线程实际扣除了I/O的时间

In [21]:
import copy


def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

1999998000000
normal:  0.13491392135620117
1999998000000
multithreading:  0.07849478721618652


## 5 lock

In [25]:
import threading

def job1():
    global A, lock
    lock.acquire()
    for i in range(10):
        A+=1
        print('job1',A)
    lock.release()

def job2():
    global A, lock
    for i in range(10):
        A+=10
        print('job2',A)

if __name__== '__main__':
    lock=threading.Lock()
    A=0
    t1=threading.Thread(target=job1)
    t2=threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110
